# Precipitation data

In [ ]:
%matplotlib inline
import os
import numpy as np
import scipy.sparse
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from itertools import combinations
from scipy import linalg
from scipy.io import loadmat
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.utils.extmath import squared_norm
from sklearn.covariance import empirical_covariance

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode()
import plotly.io as pio

from regain.utils import error_norm

In [ ]:
df_sensor = pd.read_csv("../../regain/data/Sensor_readings__with_temperature__light__humidity_every_5_minutes_at_8_locations__trial__2014_to_2015_.csv")
df_sensor = df_sensor[df_sensor['location'] == 'Docklands Library']

data_sensor = df_sensor.set_index("timestamp")[['temp_avg', 'light_avg', 'humidity_avg']]
datetime_index = pd.to_datetime(data_sensor.index)
datetime_index_hours = datetime_index.to_period('H')

data_sensor.index = datetime_index_hours
data_sensor = data_sensor[(data_sensor.index > "2014-12-15") & (data_sensor.index < "2015-01-02")]

y = data_sensor.index.to_timestamp() # data_sensor.index
X = data_sensor.values

In [ ]:
names_col = [x[0].capitalize() for x in data_sensor.columns.str.split('_')]
names_col = [(x if x != 'Humidity' else 'Hum') for x in names_col]

layout = {
        'xaxis': {
#         'tickformat': "%b '%y",
        'range' : ["2014-12-15", "2015-01-02"],
#         'tickmode': 'linear', 
#         'dtick': 86400000.0 * 100
    },
    'yaxis' : dict(
        autorange=True,
        title="Sensor readings"
    ),
    'width':1000,
    'height':500,
    'margin': {
    'l': 60,
    'r': 10,
#     'b': 5,
    't': 10,
    'pad': 0
  },
    'legend': dict(x=0, y=1.05, orientation="h")
}
fig = dict(data=[go.Scatter(
    x=y, y=data_sensor[c], name=names_col[i]) for i, c in enumerate(data_sensor.columns)],
           layout=layout)
py.iplot(fig)

In [ ]:
# from plotly import tools
# fig = tools.make_subplots(rows=3, cols=1, specs=[[{}], [{}], [{}]],
#                           shared_xaxes=True, shared_yaxes=True,
#                           vertical_spacing=0.001)
# for i, c in enumerate(data_sensor.columns) :
#     fig.append_trace(go.Scatter(x=y, y=data_sensor[c], name=names_col[i]), i+1, 1)

# layout['showlegend'] = False
# fig['layout'].update(**layout)
# fig['layout']['yaxis'].update(title=names_col[0])
# fig['layout']['yaxis2'].update(title=names_col[1])
# fig['layout']['yaxis3'].update(title=names_col[2])
# py.iplot(fig, filename='stacked-subplots-shared-xaxes')

# pio.write_image(fig, "precipitation_time_series.pdf")

In [ ]:
yy = y[0] - y
labels = (yy / np.timedelta64(1, 'h')).values

from regain.covariance import kernel_latent_time_graphical_lasso_
from sklearn.gaussian_process import kernels
kltgl = kernel_latent_time_graphical_lasso_.KernelLatentTimeGraphicalLasso(
#     alpha=0.1, tau=0.05,
    alpha=1, tau=1,
    kernel_phi=kernels.ExpSineSquared, ker_phi_param=24,
    kernel_psi=kernels.ExpSineSquared, ker_psi_param=24,
    verbose=2,
    max_iter=100).fit(X, labels)

In [ ]:
from regain.utils import save_pickle, load_pickle

# save_pickle(kltgl, "kltgl_precipitation.pkl")
# save_pickle(kltgl, "kltgl_real_alpha0.2_tau0.2.pkl")
kltgl = load_pickle("kltgl_precipitation.pkl")

In [ ]:
import plotly.io as pio
pio.orca.config.use_xvfb = True
pio.orca.config.save()

In [ ]:
import matplotlib as mpl
cmap = [x['color'] for x in list(mpl.rcParams['axes.prop_cycle'])[:3]]

data = [go.Scatter(x=sorted(y.unique()),
                   y=kltgl.precision_[:,c[0],c[1]],
                   name="%s-%s" % (names_col[c[0]], names_col[c[1]]),
    line=dict(color=cmap[(int(c[0]) + int(c[1]))-1]),
    showlegend=True)
        for c in combinations(range(len(data_sensor.columns)), 2)]

layout = dict(
    height=350,
    showlegend=True,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(
        showgrid=True,
        range=["2014-12-29", "2015-01-01"],
        nticks=20
    ),
    yaxis=dict(showgrid=True, title='Inverse Covariance'),
    margin= dict(
    l=60,
    r=10,
    b=0,
    t=10,
    pad=4)
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

In [ ]:
pio.write_image(fig, "precipitation_precision.pdf")

In [ ]:
fig = tools.make_subplots(rows=3, cols=1, specs=[[{}], [{}], [{}]],
                          shared_xaxes=True,shared_yaxes=True, vertical_spacing=0.001)

data = [go.Scatter(
    x=sorted(y.unique()),
    y=np.array([linalg.pinvh(x) for x in kltgl.precision_ - kltgl.latent_])[:,c[0],c[1]],
    name="%s-%s" % (names_col[c[0]], names_col[c[1]]),
    line=dict(color=cmap[(int(c[0]) + int(c[1]))-1]),
    showlegend=True)
        for c in combinations(range(len(data_sensor.columns)), 2)]
for i, d in enumerate(data):
    fig.append_trace(d, 1, 1)
    
data = [go.Scatter(x=sorted(y.unique()),
                   y=kltgl.precision_[:,c[0],c[1]],
                   name="%s-%s" % (names_col[c[0]], names_col[c[1]]),
    line=dict(color=cmap[(int(c[0]) + int(c[1]))-1]),
    showlegend=False)
        for c in combinations(range(len(data_sensor.columns)), 2)]
for i, d in enumerate(data):
    fig.append_trace(d, 2, 1)
    
data = [go.Scatter(x=sorted(y.unique()),
                   y=kltgl.latent_[:,c[0],c[1]],
                   name="%s-%s" % (names_col[c[0]], names_col[c[1]]),
    line=dict(color=cmap[(int(c[0]) + int(c[1]))-1]),
    showlegend=False)
        for c in combinations(range(len(data_sensor.columns)), 2)]
for i, d in enumerate(data):
    fig.append_trace(d, 3, 1)

layout['showlegend'] = True
fig['layout'].update(**layout)
fig['layout'].update(height=1500)
fig['layout']['yaxis'].update(title='Covariance')
fig['layout']['yaxis2'].update(title='Precision')#,range=[-0.099,0.15])
fig['layout']['yaxis3'].update(title='Latent contribution')#,range=[-0.1,0.075])

layout['legend']['y'] = .99
# fig = dict(data=data, layout=layout)
py.iplot(fig)

# pio.write_image(fig, "real_all_good.pdf")

# data = [go.Scatter(x=sorted(y.unique()),
#                    y=kltgl.latent_[:,c[0],c[1]],
#                    name="%s-%s" % (names_col[c[0]], names_col[c[1]]))
#         for c in combinations(range(len(data_sensor.columns)), 2)]
# layout['yaxis']["title"] = "Latent contribution"
# layout['legend']['y'] = 1.0
# fig = dict(data=data, layout=layout)
# py.iplot(fig)

# pio.write_image(fig, "real_latent.pdf")